In [111]:
import collections
import numpy as np
import pandas as pd
import re
import glob
import json

from argparse import Namespace

from torchtext.utils import download_from_url, extract_archive

In [54]:
dataset_tar = download_from_url('https://drive.google.com/uc?id=1k7GfVRqrHFK00ABkit0oGQo62fCakMSZ', root='.data/')
extracted_files = extract_archive(dataset_tar)

In [112]:
args = Namespace(
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv=".data/faq_with_splits.csv",
    output_processed_csv_prefix=".data/faq_with_splits_",
    seed=1234,
    min_questions_per_category=3000
)

In [123]:
def find_files(path):
    return glob.glob(path)

def read_json(filename):
    with open(filename, encoding='utf-8', errors='ignore') as json_file:
        data = json.load(json_file)
    return data

In [125]:
frames = []

for file in find_files('.data/gyakori_*'):
    print("Reading file:", file)
    frames.append(pd.DataFrame(read_json(file)))
    
faq = pd.concat(frames).reset_index(drop=True)

Reading file: .data/gyakori_szamitastechnika
Reading file: .data/gyakori_egeszseg
Reading file: .data/gyakori_egeszseg_20000
Reading file: .data/gyakori_allatok_14000
Reading file: .data/gyakori_szorakozas_30000


In [128]:
faq.rename(columns = {
        "valasz": "answer",
        "kategoriak": "categories",
        "hosszu_kerdes": "long_question",
        "rovid_kerdes": "short_question"
    }, inplace = True)

faq['main_category'] = faq.apply(lambda row: row.categories[0], axis = 1)
faq['sub_category'] = faq.apply(lambda row: row.categories[1], axis = 1)

faq.drop('categories', axis = 1, inplace = True)

In [129]:
faq.head()

,answer,long_question,short_question,keywords,main_category,sub_category
0,Várak régen is voltak. Ha mindhárom tornyot le...,Miért lett ilyen sz@r a honfoglaló? Régen tök ...,Miért lett ilyen sz@r a honfoglaló?,"[Honfoglaló, vár]",Számítástechnika,Internet
1,"Ahogy írták, az stdio az a C-s függvénykönyvtá...",C++-ban melyiket érdemesebb használni? Stdio v...,C++-ban melyiket érdemesebb használni? Stdio v...,"[C++, iostream, konzol, Windows, Visual Studio]",Számítástechnika,Programozás
2,"c++-ban olyat használsz, amilyet akarsz.",Lehet-e más programozási nyelvben megírt alapp...,Lehet-e más programozási nyelvben megírt alapp...,"[Java, programozás, C++]",Számítástechnika,Programozás
3,"Minden megfelel, amin fut az általad használni...",A 2016os Macbook Air megfelel egy programozást...,A 2016os Macbook Air megfelel egy programozást...,"[MacBook Air, egyetem, programozás]",Számítástechnika,Hardverek
4,"Felejtsd el,ennyiért nem kapni normálisat,vagy...",Mi a legjobb számítógépház? 10000ft körül lehe...,Mi a legjobb számítógépház?,"[AMD, Intel, PC, Radeon, NVIDIA]",Számítástechnika,Hardverek


In [130]:
categories = set(faq.main_category)
print(categories)

{'Szórakozás', 'Gyerekvállalás, nevelés', 'Állatok', 'Számítástechnika', 'Egészség'}


In [131]:
for category in categories:
    category_size = faq[faq.main_category == category].shape[0]
    if category_size < args.min_questions_per_category:
        faq = faq[faq.main_category != category]

In [132]:
def add_splits(df):
    by_main_category = collections.defaultdict(list)

    for _, row in df.iterrows():
        by_main_category[row.main_category].append(row.to_dict())
        
    final_list = []
    np.random.seed(args.seed)

    for _, item_list in sorted(by_main_category.items()):
        np.random.shuffle(item_list)
        n = len(item_list)
        n_train = int(args.train_proportion * n)
        n_val = int(args.val_proportion * n)
        n_test = int(args.test_proportion * n)

        for item in item_list[:n_train]:
            item['split'] = 'train'
        for item in item_list[n_train:n_train+n_val]:
            item['split'] = 'val'
        for item in item_list[n_train+n_val:]:
            item['split'] = 'test'  

        final_list.extend(item_list)
    return final_list

In [133]:
list_with_splits = add_splits(faq)

In [134]:
faq_with_splits = pd.DataFrame(list_with_splits)

In [135]:
faq_with_splits.head()

,answer,long_question,short_question,keywords,main_category,sub_category,split
0,"Öreg.14 èves korom òta dolgozok,hogy amit a cs...",Írnátok nekem valami biztatót? Miután pár napj...,Írnátok nekem valami biztatót?,"[depresszió, gödör, szerelem, bánat]",Egészség,Mentális egészség,train
1,"2-esre ne hallgass.Segíteni fog, az AD másképp...",Vehetek be az antidepresszánssal egy időben ny...,Vehetek be az antidepresszánssal egy időben ny...,"[antidepresszáns, tea, nyugtató, gyógynövény]",Egészség,Mentális egészség,train
2,"Csak a hülyeséggel nem kell feküdni, azt lábon...",Azt halottam tüdőgyulladással nem nagyon kell ...,Azt halottam tüdőgyulladással nem nagyon kell ...,"[tüdőgyulladás, vesegyulladás]",Egészség,Betegségek,train
3,"Ignoráld. A közösségek ilyenek, kinéznek valak...","Mit csináljak, ha szekál az osztályom a miatt,...","Mit csináljak, ha szekál az osztályom a miatt,...","[szekálás, betegség, pajzsmirigy, hiányzás]",Egészség,"Immunrendszer, fertőzések",train
4,Kedves kérdező!!Először is nem tudom hány éves...,"Hirtelen nagyon erős lett a vérzésem, egyik pe...","Hirtelen nagyon erős lett a vérzésem, egyik pe...","[vérzés, darabos, nőgyógyász]",Egészség,Nők egészsége,train


In [139]:
from unidecode import unidecode

def unicode_to_ascii(data):
    return unidecode(re.sub(r"[,.;@#?!&$]+\ *", " ", data).lower(), errors='ignore')

ascii_faq = faq_with_splits.copy()

ascii_faq.long_question = ascii_faq.long_question.apply(lambda x: unicode_to_ascii(x))
ascii_faq.short_question = ascii_faq.short_question.apply(lambda x: unicode_to_ascii(x))
ascii_faq.answer = ascii_faq.answer.apply(lambda x: unicode_to_ascii(x))

In [140]:
ascii_faq.head()

,answer,long_question,short_question,keywords,main_category,sub_category,split
0,oreg 14 eves korom ota dolgozok hogy amit a cs...,irnatok nekem valami biztatot miutan par napja...,irnatok nekem valami biztatot,"[depresszió, gödör, szerelem, bánat]",Egészség,Mentális egészség,train
1,2-esre ne hallgass segiteni fog az ad maskepp ...,vehetek be az antidepresszanssal egy idoben ny...,vehetek be az antidepresszanssal egy idoben ny...,"[antidepresszáns, tea, nyugtató, gyógynövény]",Egészség,Mentális egészség,train
2,csak a hulyeseggel nem kell fekudni azt labon ...,azt halottam tudogyulladassal nem nagyon kell ...,azt halottam tudogyulladassal nem nagyon kell ...,"[tüdőgyulladás, vesegyulladás]",Egészség,Betegségek,train
3,ignorald a kozossegek ilyenek kineznek valakit...,mit csinaljak ha szekal az osztalyom a miatt h...,mit csinaljak ha szekal az osztalyom a miatt h...,"[szekálás, betegség, pajzsmirigy, hiányzás]",Egészség,"Immunrendszer, fertőzések",train
4,kedves kerdezo eloszor is nem tudom hany eves ...,hirtelen nagyon eros lett a verzesem egyik per...,hirtelen nagyon eros lett a verzesem egyik per...,"[vérzés, darabos, nőgyógyász]",Egészség,Nők egészsége,train


In [141]:
ascii_faq.split.value_counts()

train    117686
test      25223
val       25217
Name: split, dtype: int64

In [142]:
ascii_faq.to_csv(args.output_munged_csv, index=False)

### Stemming

In [155]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import HungarianStemmer
from nltk.corpus import stopwords

import nltk

import hu_core_ud_lg

In [156]:
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('hungarian'))

In [157]:
tokenizer = RegexpTokenizer(r'\w+')

stemmer = HungarianStemmer()

question = faq_with_splits.iloc[2]['long_question']

# Tokenization
tokenized_sentence = tokenizer.tokenize(question)

# Stopword filtering
filtered_sentence = [w for w in tokenized_sentence if not w in stop_words]

# Lemmatization
lemmatizer = hu_core_ud_lg.load()

lemmatizer.remove_pipe('parser')
lemmatizer.remove_pipe('ner')
lemmatizer.add_pipe(lemmatizer.create_pipe('sentencizer'))

doc = lemmatizer(" ".join(filtered_sentence))

In [158]:
print(f"Original question:\n{question}\n")

print("After tokenization:")
for w in tokenized_sentence:
    print(w, end=" ")

print("\n\nAfter stopword filtering:")
for w in filtered_sentence:
    print(w, end=" ")

print("\n\nAfter lemmatization:")
for w in doc:
    print(w.lemma_.lower(), end=" ")
    
print("\n\nAfter stemming:")
for w in tokenized_sentence:
    print(stemmer.stem(w), end=" ")

Original question:
Azt halottam tüdőgyulladással nem nagyon kell feküdni. Vese gyulladással pedig igen. Mivel sok vizelet távozik a másodikra gondoltam. Fő tünet bal oldalon középtájt fáj a hátam. Melyik lehet? Egész nap a fokozott vizeletürítésen kivűl bajom nincs, félig fekvő helyzetben pihenve 1 2 óra után megfájdúl.

After tokenization:
Azt halottam tüdőgyulladással nem nagyon kell feküdni Vese gyulladással pedig igen Mivel sok vizelet távozik a másodikra gondoltam Fő tünet bal oldalon középtájt fáj a hátam Melyik lehet Egész nap a fokozott vizeletürítésen kivűl bajom nincs félig fekvő helyzetben pihenve 1 2 óra után megfájdúl 

After stopword filtering:
Azt halottam tüdőgyulladással feküdni Vese gyulladással Mivel vizelet távozik másodikra gondoltam Fő tünet bal oldalon középtájt fáj hátam Melyik Egész nap fokozott vizeletürítésen kivűl bajom félig fekvő helyzetben pihenve 1 2 óra megfájdúl 

After lemmatization:
az halik tüdőgyulladás fekszik vese gyulladás mivel vizelet távozik 

In [159]:
import time

In [160]:
def lemmatize_question(question):
    doc = lemmatizer(question)
    lemmatized = []
    
    for w in doc:
        lemmatized.append(w.lemma_.lower())
    return " ".join(lemmatized)

In [161]:
def stem_question(question):
    stemmed = []
    
    for w in question.split():
        stemmed.append(stemmer.stem(w))
    return " ".join(stemmed)

In [162]:
def filter_question(question):
    tokenized_question = " ".join(tokenizer.tokenize(question))
    filtered = [w for w in tokenized_question.split() if not w in stop_words]

    return " ".join(filtered)

In [163]:
def create_dataset(questions, func, name):
    result = []

    start_time = int(time.time() * 1000)
    
    for idx, q in enumerate(questions):
        result.append(func(q))
    
    end_time = int(time.time() * 1000)
    print(f"Creating dataset called '{name}' took: {(end_time - start_time) / 1000.0} seconds")
    
    return result

In [164]:
def create_datasets(questions):
    result = {}
    
    result['stemmed'] = create_dataset(questions, stem_question, "stemmed")
    result['lemmatized'] = create_dataset(questions, lemmatize_question, "lemmatized")
    
    filtered_questions = create_dataset(questions, filter_question, "filtered")
    
    result['stemmed_filtered'] = create_dataset(filtered_questions, stem_question, "stemmed_filtered")
    result['lemmatized_filtered'] = create_dataset(filtered_questions, lemmatize_question, "lemmatized_filtered")
    
    return result

In [165]:
questions = faq_with_splits['short_question'].tolist()

In [166]:
datasets = create_datasets(questions)

Creating dataset called 'stemmed' took: 15.619 seconds
Creating dataset called 'lemmatized' took: 234.092 seconds
Creating dataset called 'filtered' took: 0.762 seconds
Creating dataset called 'stemmed_filtered' took: 11.369 seconds
Creating dataset called 'lemmatized_filtered' took: 208.98 seconds


In [167]:
dataframes = {}
for dataset_key in datasets:
    dataframe = faq_with_splits.copy()
    
    dataframe['short_question'] = datasets[dataset_key]
    
    dataframes[dataset_key] = dataframe

In [168]:
dataframes['stemmed'].head()

,answer,long_question,short_question,keywords,main_category,sub_category,split
0,"Öreg.14 èves korom òta dolgozok,hogy amit a cs...",Írnátok nekem valami biztatót? Miután pár napj...,írnát nek valam biztatót?,"[depresszió, gödör, szerelem, bánat]",Egészség,Mentális egészség,train
1,"2-esre ne hallgass.Segíteni fog, az AD másképp...",Vehetek be az antidepresszánssal egy időben ny...,vehet be az antidepresszáns egy idő nyugtatót?,"[antidepresszáns, tea, nyugtató, gyógynövény]",Egészség,Mentális egészség,train
2,"Csak a hülyeséggel nem kell feküdni, azt lábon...",Azt halottam tüdőgyulladással nem nagyon kell ...,az halott tüdőgyulladás nem nagy kell feküdni....,"[tüdőgyulladás, vesegyulladás]",Egészség,Betegségek,train
3,"Ignoráld. A közösségek ilyenek, kinéznek valak...","Mit csináljak, ha szekál az osztályom a miatt,...","mi csináljak, ha szekál az osztály a miatt, ho...","[szekálás, betegség, pajzsmirigy, hiányzás]",Egészség,"Immunrendszer, fertőzések",train
4,Kedves kérdező!!Először is nem tudom hány éves...,"Hirtelen nagyon erős lett a vérzésem, egyik pe...","hirtel nagy erős let a vérzésem, egy percről a...","[vérzés, darabos, nőgyógyász]",Egészség,Nők egészsége,train


In [169]:
dataframes['lemmatized'].head()

,answer,long_question,short_question,keywords,main_category,sub_category,split
0,"Öreg.14 èves korom òta dolgozok,hogy amit a cs...",Írnátok nekem valami biztatót? Miután pár napj...,írnátok nek valami biztató ?,"[depresszió, gödör, szerelem, bánat]",Egészség,Mentális egészség,train
1,"2-esre ne hallgass.Segíteni fog, az AD másképp...",Vehetek be az antidepresszánssal egy időben ny...,vehet be az antidepresszáns egy idő nyugtató ?,"[antidepresszáns, tea, nyugtató, gyógynövény]",Egészség,Mentális egészség,train
2,"Csak a hülyeséggel nem kell feküdni, azt lábon...",Azt halottam tüdőgyulladással nem nagyon kell ...,az halik tüdőgyulladás nem nagyon kell fekszik...,"[tüdőgyulladás, vesegyulladás]",Egészség,Betegségek,train
3,"Ignoráld. A közösségek ilyenek, kinéznek valak...","Mit csináljak, ha szekál az osztályom a miatt,...","mi csinál , ha szekál az osztály a miatt , hog...","[szekálás, betegség, pajzsmirigy, hiányzás]",Egészség,"Immunrendszer, fertőzések",train
4,Kedves kérdező!!Először is nem tudom hány éves...,"Hirtelen nagyon erős lett a vérzésem, egyik pe...","hirtelen nagyon erős lett a vérzésem , egyik p...","[vérzés, darabos, nőgyógyász]",Egészség,Nők egészsége,train


In [170]:
dataframes['stemmed_filtered'].head()

,answer,long_question,short_question,keywords,main_category,sub_category,split
0,"Öreg.14 èves korom òta dolgozok,hogy amit a cs...",Írnátok nekem valami biztatót? Miután pár napj...,írnát biztató,"[depresszió, gödör, szerelem, bánat]",Egészség,Mentális egészség,train
1,"2-esre ne hallgass.Segíteni fog, az AD másképp...",Vehetek be az antidepresszánssal egy időben ny...,vehet antidepresszáns idő nyugtató,"[antidepresszáns, tea, nyugtató, gyógynövény]",Egészség,Mentális egészség,train
2,"Csak a hülyeséggel nem kell feküdni, azt lábon...",Azt halottam tüdőgyulladással nem nagyon kell ...,az halott tüdőgyulladás feküdn ves gyulladás m...,"[tüdőgyulladás, vesegyulladás]",Egészség,Betegségek,train
3,"Ignoráld. A közösségek ilyenek, kinéznek valak...","Mit csináljak, ha szekál az osztályom a miatt,...",mi csinálj ha szekál osztály miat beteg hiányz,"[szekálás, betegség, pajzsmirigy, hiányzás]",Egészség,"Immunrendszer, fertőzések",train
4,Kedves kérdező!!Először is nem tudom hány éves...,"Hirtelen nagyon erős lett a vérzésem, egyik pe...",hirtel erős vérzés percről más ömlen kezdet ré...,"[vérzés, darabos, nőgyógyász]",Egészség,Nők egészsége,train


In [171]:
dataframes['lemmatized_filtered'].head()

,answer,long_question,short_question,keywords,main_category,sub_category,split
0,"Öreg.14 èves korom òta dolgozok,hogy amit a cs...",Írnátok nekem valami biztatót? Miután pár napj...,írnátok biztató,"[depresszió, gödör, szerelem, bánat]",Egészség,Mentális egészség,train
1,"2-esre ne hallgass.Segíteni fog, az AD másképp...",Vehetek be az antidepresszánssal egy időben ny...,vehet antidepresszáns idő nyugtató,"[antidepresszáns, tea, nyugtató, gyógynövény]",Egészség,Mentális egészség,train
2,"Csak a hülyeséggel nem kell feküdni, azt lábon...",Azt halottam tüdőgyulladással nem nagyon kell ...,az halik tüdőgyulladás fekszik vese gyulladás ...,"[tüdőgyulladás, vesegyulladás]",Egészség,Betegségek,train
3,"Ignoráld. A közösségek ilyenek, kinéznek valak...","Mit csináljak, ha szekál az osztályom a miatt,...",mi csinál ha szekál osztály miatt beteg hiányzik,"[szekálás, betegség, pajzsmirigy, hiányzás]",Egészség,"Immunrendszer, fertőzések",train
4,Kedves kérdező!!Először is nem tudom hány éves...,"Hirtelen nagyon erős lett a vérzésem, egyik pe...",hirtelen erős vérzésem perc másikra ömlik kezd...,"[vérzés, darabos, nőgyógyász]",Egészség,Nők egészsége,train


In [172]:
dataframes['stemmed_filtered'].head()

,answer,long_question,short_question,keywords,main_category,sub_category,split
0,"Öreg.14 èves korom òta dolgozok,hogy amit a cs...",Írnátok nekem valami biztatót? Miután pár napj...,írnát biztató,"[depresszió, gödör, szerelem, bánat]",Egészség,Mentális egészség,train
1,"2-esre ne hallgass.Segíteni fog, az AD másképp...",Vehetek be az antidepresszánssal egy időben ny...,vehet antidepresszáns idő nyugtató,"[antidepresszáns, tea, nyugtató, gyógynövény]",Egészség,Mentális egészség,train
2,"Csak a hülyeséggel nem kell feküdni, azt lábon...",Azt halottam tüdőgyulladással nem nagyon kell ...,az halott tüdőgyulladás feküdn ves gyulladás m...,"[tüdőgyulladás, vesegyulladás]",Egészség,Betegségek,train
3,"Ignoráld. A közösségek ilyenek, kinéznek valak...","Mit csináljak, ha szekál az osztályom a miatt,...",mi csinálj ha szekál osztály miat beteg hiányz,"[szekálás, betegség, pajzsmirigy, hiányzás]",Egészség,"Immunrendszer, fertőzések",train
4,Kedves kérdező!!Először is nem tudom hány éves...,"Hirtelen nagyon erős lett a vérzésem, egyik pe...",hirtel erős vérzés percről más ömlen kezdet ré...,"[vérzés, darabos, nőgyógyász]",Egészség,Nők egészsége,train


In [173]:
for dataframe_key in dataframes:
    filename = f"{args.output_processed_csv_prefix}{dataframe_key}.csv"
    print(f"Creating {filename}")
    dataframes[dataframe_key].to_csv(filename, index=False, encoding='utf-8', errors='ignore')

Creating .data/faq_with_splits_stemmed.csv
Creating .data/faq_with_splits_lemmatized.csv
Creating .data/faq_with_splits_stemmed_filtered.csv
Creating .data/faq_with_splits_lemmatized_filtered.csv
